# 1.0 Install and Import the Required Libraries

In [1]:
# pip install kfp
# !pip install watermark

In [3]:
# IMPORT REQUIRED LIBRARIES
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        Markdown,
                        HTML,
                        component, 
                        OutputPath, 
                        InputPath)
from kfp import compiler
from google.cloud.aiplatform import pipeline_jobs

import kfp
import google.cloud.aiplatform 
print(f"kfp version: {kfp.__version__}")
print(f"google.cloud.aiplatform version: {google.cloud.aiplatform.__version__}")

kfp version: 2.9.0
google.cloud.aiplatform version: 1.71.1


# 2.0 Define the Components

## 2.1 Data Cleaning

In [4]:
# Set the docker image 
BASE_IMAGE = "eu.gcr.io/norse-voice-440615-v3/practice_gcp@sha256:506a11f1f5ded3bc2c4fb97f560e7aedf91a89dcc53f4b730c9ca4b8de6a777d"

@component(
    base_image=BASE_IMAGE, # Load the docker image
    output_component_file="data_cleaning.yaml" # Output filename
)
def data_cleaning(
    output_dir: Output[Dataset], # Output path
):

    from src.preprocess import data_cleaning
    
    # Clean the data
    df = data_cleaning()
    print("Data is cleaned")
    
    # Save dataframe as CSV
    df.to_csv(output_dir.path, index=False)

C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1102930185.py:3: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1102930185.py:7: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def data_cleaning(


## 2.2 Data Processing
- Feature Engineering 
- Label Encoding

In [5]:
@component(
    base_image=BASE_IMAGE, # Load the docker image
    output_component_file="data_processing.yaml", # Output filename
)
def data_processing(
    dataset_full: Input[Dataset], # Input the cleaned dataset
    dataset_processed: Output[Dataset]): # Output processed dataset

    import pandas as pd
    from src.preprocess import data_preprocessing

    df = pd.read_csv(dataset_full.path)
    
    # Process the dataframe
    df_processed = data_preprocessing(df)
    print("Performed Feature Engineering and Label Encoding")
    
    # Save processed dataframe as CSV
    df_processed.to_csv(dataset_processed.path, index=False)

C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\317081645.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\317081645.py:5: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def data_processing(


## 2.3 Model Training

In [6]:
@component(
    base_image=BASE_IMAGE, # Load the docker image
    output_component_file="train_model.yaml", # Output filename
)
def train_model(
    dataset_processed: Input[Dataset], # Input processed dataset
    model: Output[Model] # Output model scores
):
    import json
    import pandas as pd
    from src.train import train_and_evaluate
    
    # Load the processed dataset
    df = pd.read_csv(dataset_processed.path)
    
    # Pass `model.path` to save the trained model to that path
    outputs = train_and_evaluate(df, model.path)
    scores = outputs['scores']
    
    # Save the evaluation scores in json file
    with open(model.path + "_scores.json", 'w') as f:
        json.dump(scores, f)

    # Print the path to the model and the model scores
    print(f"Model artifact saved at: {model.path}")
    print(f"Scores saved at: {model.path}_scores.json")

C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1835865731.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1835865731.py:5: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def train_model(


## 2.4 Model Evaluation

In [7]:
@component(
    base_image=BASE_IMAGE, # Load the docker image
    output_component_file="compute_metrics.yaml", # Output filename
)
def compute_metrics(
    model: Input[Model], # Input the trained model
    train_metric: Output[Metrics], # Output training score
    test_metric: Output[Metrics], # Output testing score
    cv_metric: Output[Metrics] # Output cross validation score (5 folds)
):
    
    import json
    
    # Define the path to the scores JSON file
    scores_file_name = model.path + "_scores.json"
    
    # Open and load the scores from the JSON file
    with open(scores_file_name, 'r') as file:  
        model_metrics = json.load(file)
        
    # Log the metrics for train, test, and cross-validation AUC
    train_metric.log_metric('train_auc', model_metrics['train'])
    test_metric.log_metric('test_auc', model_metrics['test'])
    cv_metric.log_metric('cv_auc', model_metrics['cross_validation (5 folds)'])


C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1926838823.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
C:\Users\Hezron Ling\AppData\Local\Temp\ipykernel_6096\1926838823.py:5: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def compute_metrics(


# 3.0 Define and Run the Pipeline

In [8]:
# Set the Google Application Credential (JSON Key)
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Hezron Ling/Desktop/Practice_GCP/norse-voice-440615-v3-e1e314d6a20e.json"

In [9]:
import datetime as dt

# Use timestamp to define the name of each pipeline
TIMESTAMP = dt.datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = 'pipeline-practice_gcp-{}'.format(TIMESTAMP)
BUCKET_NAME = 'norse-voice-440615-v3_cloudbuild'
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root/"
REGION = "asia-southeast2"
PROJECT_ID = "norse-voice-440615-v3"

# Define the pipeline, each component reuses outputs from previous component
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline
    name="pipeline-practiceGCP"   
)
def pipeline(
    project: str = PROJECT_ID, 
    region: str = REGION, 
    display_name: str = DISPLAY_NAME
):
    
    # Data cleaning
    data_cleaning_op = data_cleaning()
    
    # Data processing
    data_processing_op = data_processing(
        dataset_full=data_cleaning_op.outputs["output_dir"]
    )
    
    # Model training
    train_model_op = train_model(
        dataset_processed=data_processing_op.outputs["dataset_processed"]
    ).set_cpu_limit('8').set_memory_limit('32G') # Increase CPU and Memory 
    
    # Model evaluation
    model_evaluation_op = compute_metrics(
        model=train_model_op.outputs["model"]
    )

# Compile the pipeline as JSON file
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='practiceGCP_pipeline.json'
)

# Start running the pipeline
start_pipeline = pipeline_jobs.PipelineJob(
    display_name="practiceGCP-pipeline",
    template_path="practiceGCP_pipeline.json",
    enable_caching=False,
    location=REGION,
    project=PROJECT_ID
)

# Run the pipeline
start_pipeline.run(service_account="120166958921-compute@developer.gserviceaccount.com")

Creating PipelineJob
PipelineJob created. Resource name: projects/120166958921/locations/asia-southeast2/pipelineJobs/pipeline-practicegcp-20241210174215
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/120166958921/locations/asia-southeast2/pipelineJobs/pipeline-practicegcp-20241210174215')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/asia-southeast2/pipelines/runs/pipeline-practicegcp-20241210174215?project=120166958921
PipelineJob projects/120166958921/locations/asia-southeast2/pipelineJobs/pipeline-practicegcp-20241210174215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/120166958921/locations/asia-southeast2/pipelineJobs/pipeline-practicegcp-20241210174215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/120166958921/locations/asia-southeast2/pipelineJobs/pipeline-practicegcp-20241210174215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob proje